In [1]:
#!/usr/bin/env python3

import os, time, sys
sys.path.append('../')
import numpy as np
import pyhepmc
import random
import logging
import configparser
import subprocess
import multiprocessing
import itertools
import progressbar as P
from helper import getModelDict
delphesDir = os.path.abspath("../DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT


ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')



Welcome to JupyROOT 6.30/06


True

In [2]:
inputFile = '../pp2chi0chi0_minimalH_scan_jets/Events/run_01/ddmH_mS_2000_m1_979_dm_90_delphes_events.root'
# inputFile = '../DelphesLLP/test.root'

In [ ]:
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
tree.GetEntry(10)

storeParticles = []

pS = np.array([0.0,0.0,0.0,0.0])
print('############ ON-TIME #####################')
print('llp:')
for p in tree.llpParticlesOnTime:
    print(p.PID,p.Status)
    print(p.Px,p.Py,p.Pz,p.E)

print('mom:')
for p in tree.llpMothersOnTime:
    print(p.PID,p.Status)    

print('direct:')
pVis = np.array([0.0,0.0,0.0,0.0])
pInv = np.array([0.0,0.0,0.0,0.0])
for p in tree.llpDirectDaughtersOnTime:
    print(p.PID,p.Status)   
    if abs(p.PID) < 10000:
        pVis += np.array([p.Px,p.Py,p.Pz,p.E])
    else:
        pInv += np.array([p.Px,p.Py,p.Pz,p.E])
pTvis = np.sqrt(pVis[0]**2+pVis[1]**2)
print(f'  (pT visible = {pTvis})')
pTot = pVis+pInv
print(f'  (pTot = {pTot})')
mTot = np.sqrt(pTot[3]**2-np.dot(pTot[:3],pTot[:3]))
print(f'  (mTot  = {mTot})')
pS = pS + pTot

print('final:')
pVis = np.array([0.0,0.0,0.0,0.0])
for p in tree.llpFinalOnTime:
    if abs(p.PID) > 10000 or p.Status != 1:
        print(p.PID,p.Status)
    else:
        pVis += np.array([p.Px,p.Py,p.Pz,p.E])
print('...')
pTvis = np.sqrt(pVis[0]**2+pVis[1]**2)
print(f'  (pT visible = {pTvis})')

print('others:')
for p in tree.llpOthersOnTime:
    if abs(p.PID) > 10000 or p.Status != 1:
        print(p.PID,p.Status)
print('...')

print('############ DELAYED #####################')
print('llp:')
for p in tree.llpParticlesDelayed:
    print(p.PID,p.Status)
    print(p.Px,p.Py,p.Pz,p.E)

print('mom:')
for p in tree.llpMothersDelayed:
    print(p.PID,p.Status)    

print('direct:')
pVis = np.array([0.0,0.0,0.0,0.0])
pInv = np.array([0.0,0.0,0.0,0.0])
for p in tree.llpDirectDaughtersDelayed:
    print(p.PID,p.Status)   
    if abs(p.PID) < 10000:
        pVis += np.array([p.Px,p.Py,p.Pz,p.E])
    else:
        pInv += np.array([p.Px,p.Py,p.Pz,p.E])
pTvis = np.sqrt(pVis[0]**2+pVis[1]**2)
print(f'  (pT visible = {pTvis})')
pTot = pVis+pInv
print(f'  (pTot = {pTot})')
mTot = np.sqrt(pTot[3]**2-np.dot(pTot[:3],pTot[:3]))
print(f'  (mTot  = {mTot})')
pS = pS + pTot

print('final:')
pVis = np.array([0.0,0.0,0.0,0.0])
for p in tree.llpFinalDelayed:
    if abs(p.PID) > 10000 or p.Status != 1:
        print(p.PID,p.Status)
    else:
        storeParticles.append([p.Px,p.Py,p.Pz,p.E])
        pVis += np.array([p.Px,p.Py,p.Pz,p.E])
print('...')
pTvis = np.sqrt(pVis[0]**2+pVis[1]**2)
print(f'  (pT visible = {pTvis})')

print('others:')
for p in tree.llpOthersDelayed:
    if abs(p.PID) > 10000 or p.Status != 1:
        print(p.PID,p.Status)
print('...')

pTS = np.sqrt(pS[0]**2 + pS[1]**2)
print(f'  (pS = {pS}, pTS = {pTS})')
mS = np.sqrt(pS[3]**2-np.dot(pS[:3],pS[:3]))
print(f'  (MS  = {mS})')


print('####### ALL JETS ################')
for j in tree.GenJet:
    print(j.PT,len(j.Constituents))


print('####### ON-TIME JETS ################')
for j in tree.GenJetOnTime:
    print(j.PT,len(j.Constituents))    



print('####### Delayed JETS ################')
for j in tree.GenJetDelayed:
    print(j.PT,len(j.Constituents))        
f.Close()

############ ON-TIME #####################
llp:
4000023 62
-460.4539489746094 -344.4481201171875 31.33608055114746 1135.81884765625
mom:
4000023 44
direct:
5 23
-5 23
4000022 1
  (pT visible = 26.947889117924923)
  (pTot = [-460.45393562 -344.44812202   31.33608055 1135.81881714])
  (mTot  = 978.9995402577514)
final:
4000022 1
...
  (pT visible = 26.947891034884087)
others:
-4000022 1
...
############ DELAYED #####################
llp:
-4000023 62
328.6488342285156 474.78594970703125 -530.1239624023438 1254.15478515625
mom:
-4000023 44
direct:
5 23
-5 23
-4000022 1
  (pT visible = 38.101932174954655)
  (pTot = [ 328.64884186  474.78593349 -530.1239388  1254.15476227])
  (mTot  = 979.0000170240813)
final:
-4000022 1
...
  (pT visible = 38.101931485393564)
others:
4000022 1
...
  (pS = [-131.80509377  130.33781147 -498.78785825 2389.97357941], pTS = 185.36592955898396)
  (MS  = 2329.9836597894)
####### ALL JETS ################
185.7950897216797 30
74.35649108886719 33
27.846254348754883

: 

In [ ]:
import fastjet
import awkward as ak
import vector
vector.register_awkward()

jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.6)
array0 = []
for p in storeParticles:
    array0.append({'px' : p[0], 'py' : p[1], 'pz' : p[2], 'E' : p[3]})
array1 = ak.Array(array0)


array1 = ak.Array(
     [
         {"px": 1.2, "py": 3.2, "pz": 5.4, "E": 23.5},
         {"px": 32.2, "py": 64.21, "pz": 543.34, "E": 755.12},
         {"px": 32.45, "py": 63.21, "pz": 543.14, "E": 835.56},
         
     ],with_name="Momentum4D",
 )
cluster = fastjet.ClusterSequence(array1, jetdef)
cluster.inclusive_jets()